# Parallelism & Concurrency
In this notebook, we will examine how we can use parallelism in our advantage and run our codes faster.
To better understand parallelism, I will cover the below topics:
- Understanding Concurrency and Parallelism
- Asyncio for Concurrency
- Threading for Concurrency
- Multiprocessing for Parallelism
- Practical Examples

## 1. Understanding Concurrency and Parallelism
At first, we should learn the difference between these terms: `concurrency` and `parallelism`:
- `Concurrency`: Concurrency is about dealing with multiple tasks at once but not necessarily doing them simultaneously. It gives the `illusion of parallelism` by switching between tasks efficiently.

- `Parallelism`: Parallelism is executing multiple tasks simultaneously, leveraging multicore processors to perform computation-heavy tasks in `parallel`.

### Note
We will use `Asyncio` for concurrency and `Threading` for parallelism:


| ----- | ---- |
| concurrency    | asyncio   |
| parallelism  | threading/multiprocessing   |

## 2. Asyncio for Concurrency
`Asyncio` is a library for asynchronous programming in Python, allowing you to write concurrent code using the `async/await` syntax. It may sound confusing but I will teach you in one go so you understand. Look at the file: `example.py`.

### 2.1. Details
There are only two words that matter: `async` and `await`.

The `async` parameter shows that your function wants to use `concurrency`. So, you just put the `async` phrase before your definition of your function just like the top example. This `async` thing, shows that it should run in concurrent mode, so any running, should have something to gather data and results.

Now for the result, the `await` comes into account. The `await` phrase, demonstrate that we should wait for the result of the answer/result of what is written after the await call:
```python
await asyncio.sleep(delay)
```
So, in this example, we should `await` for the answer of `asyncio.sleep(delay)`.

### 2.2. Asyncio Queue
As you may have guessed, when we have some tasks/jobs, we need a queue and a distributer to make the decision what should be run now and what is the next job/task. In order to do so, we use `asyncio.run()` function to state that we want these functions to be run.


### 2.3. asyncio.run()
`asyncio.run()` is a high-level function used to run an asynchronous function (coroutine) from a synchronous context. It sets up the event loop, runs the coroutine, and handles shutting down the event loop when the coroutine completes.

#### Example
```python
import asyncio

async def main():
    print("Hello, World!")

# Running the main coroutine
asyncio.run(main())
```

### 2.4. asyncio.gather()
`asyncio.gather()` is used to run multiple coroutines concurrently and aggregate their results. It takes multiple coroutine objects and returns a single coroutine that gathers the results of those coroutines into a list.

#### Example
```python
import asyncio

async def say_after(delay, message):
    await asyncio.sleep(delay)
    return message

async def main():
    result = await asyncio.gather(
        say_after(1, 'Hello'),
        say_after(2, 'World')
    )
    print(result)  # Output: ['Hello', 'World']

asyncio.run(main())
```

### 2.5. asyncio.create_task()
Schedules the execution of a coroutine object in the event loop. Returns an asyncio.Task object.

#### Example
```python
import asyncio

async def say_hello():
    await asyncio.sleep(1)
    print("Hello")

async def main():
    task = asyncio.create_task(say_hello())
    await task

asyncio.run(main())
```

### asyncio.sleep()
Asynchronously sleeps for a specified number of seconds, allowing other tasks to run during this period.

#### Example
```python
import asyncio

async def main():
    print("Sleeping...")
    await asyncio.sleep(2)
    print("Awake!")

asyncio.run(main())
```

### asyncio.wait()
Waits for the completion of multiple `Future` or `Task` objects. Can be configured to return when any or all of the objects are done.


#### Example
```python
import asyncio

async def say_hello():
    await asyncio.sleep(1)
    return "Hello"

async def say_world():
    await asyncio.sleep(2)
    return "World"

async def main():
    tasks = [say_hello(), say_world()]
    done, pending = await asyncio.wait(tasks)
    for task in done:
        print(task.result())

asyncio.run(main())
```

### asyncio.as_completed()
Returns an iterator of `Future` objects that yield results as they complete.

#### Example
```python
import asyncio

async def say_hello():
    await asyncio.sleep(1)
    return "Hello"

async def say_world():
    await asyncio.sleep(2)
    return "World"

async def main():
    tasks = [say_hello(), say_world()]
    for task in asyncio.as_completed(tasks):
        result = await task
        print(result)

asyncio.run(main())
```

### asyncio.shield()

Protects a coroutine from being cancelled.

#### Example
```python
import asyncio

async def protected_task():
    await asyncio.sleep(3)
    return "Completed"

async def main():
    task = asyncio.create_task(asyncio.shield(protected_task()))
    await asyncio.sleep(1)
    task.cancel()
    try:
        await task
    except asyncio.CancelledError:
        print("Task was not cancelled")

asyncio.run(main())
```

### asyncio.Event
A synchronization primitive that allows one or more coroutines to wait until an event is set.

#### Example
```python
import asyncio

async def waiter(event):
    print("Waiting for event to be set")
    await event.wait()
    print("Event is set")

async def main():
    event = asyncio.Event()
    asyncio.create_task(waiter(event))
    await asyncio.sleep(2)
    event.set()

asyncio.run(main())
```

### asyncio.Queue
An asynchronous `FIFO queue` used to exchange data between coroutines.

#### Example
```python
import asyncio

async def producer(queue):
    for i in range(5):
        await asyncio.sleep(1)
        await queue.put(i)
        print(f"Produced {i}")

async def consumer(queue):
    while True:
        item = await queue.get()
        print(f"Consumed {item}")
        queue.task_done()

async def main():
    queue = asyncio.Queue()
    await asyncio.gather(producer(queue), consumer(queue))

asyncio.run(main())
```